In [1]:
# import libraries 
import nest
import nest.voltage_trace
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import pandas as pd
import networkx as nx
from nest.lib.hl_api_types import NodeCollection


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.8.0
 Built: Aug 27 2024 04:36:56

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [2]:
# load connectivity matrices
aa = pd.read_csv('../Datasets/Original/aa_connectivity_matrix.csv', index_col=0)
ad = pd.read_csv('../Datasets/Original/ad_connectivity_matrix.csv', index_col=0)
da = pd.read_csv('../Datasets/Original/da_connectivity_matrix.csv', index_col=0)
dd = pd.read_csv('../Datasets/Original/dd_connectivity_matrix.csv', index_col=0)

In [3]:
# build graph used for visualization and add nodes
G = nx.DiGraph()
for neuron_id in aa.index:
    G.add_node(int(neuron_id))
    
# add weighted edges if any connection exists (weight > 0)
# sum weights from all 4 connection types (aa, ad, da, dd)
for pre_id in aa.index:
    for post_id in aa.columns:
        weight = aa.loc[pre_id, post_id] + ad.loc[pre_id, post_id] + da.loc[pre_id, post_id] + dd.loc[pre_id, post_id]
        if weight > 0:
            G.add_edge(int(pre_id), int(post_id), weight=(weight))

In [4]:
# load and merge neuron attributes from CSVs
attr3 = pd.read_csv('../Datasets/Original/s3.csv')
attr4 = pd.read_csv('../Datasets/Original/s4.csv')

In [5]:
all_attrs = attr3.merge(attr4, on='skid', how='left', suffixes=('_axon', '_dendrite'))
print(all_attrs.head())  

# assign these attributes to graph 
for _, row in all_attrs.iterrows():
    nid = row['skid']  
    if nid in G.nodes:
        G.nodes[nid]['cell_type'] = row.get('celltype_axon', row.get('celltype', None))
        G.nodes[nid]['axonIO_ratio'] = row.get('axonIO_ratio', None)
        G.nodes[nid]['dendritic_ratio'] = row.get('dendritic_output-input_ratio', None)


       skid celltype_axon  axonIO_ratio celltype_dendrite  \
0   7616956         MBINs      3.162791               NaN   
1   7983899         MBINs      2.652941             MBINs   
2   8318678           KCs      1.710526               KCs   
3  10673895         MBINs      1.615044               NaN   
4   4381377         MBINs      1.500000             MBINs   

   dendritic_output-input_ratio  
0                           NaN  
1                      0.246733  
2                      0.595238  
3                           NaN  
4                      0.004687  


In [41]:
# reset the NEST kernel before simulation
nest.ResetKernel()

In [42]:
neuron_params = {
    "C_m": 250.0,        # мембранная емкость 
    "tau_m": 20.0,       # временная константа 
    "t_ref": 5.0,        # рефрактерный период 
    "E_L": -70.0,        # потенциал покоя 
    "V_reset": -70.0,    # потенциал после спайка 
    "V_th": -55.0        # порог 
}

In [43]:
N_neurons = G.number_of_nodes()
N_neurons

2952

In [44]:
# create neurons
neurons = nest.Create("iaf_psc_alpha", N_neurons, params=neuron_params)
# mapping between graph nodes and neuron list
node_list = list(G.nodes)
node_index = {node: i for i, node in enumerate(node_list)}

In [45]:
neurons

NodeCollection(metadata=None, model=iaf_psc_alpha, size=2952, first=1, last=2952)

In [46]:
# connect neurons based on connectivity matrices, 
# for each conn type, create a connection if weight > 0 
for pre_id in aa.index:
    for post_id in aa.columns:
        # get the index in the NEST neuron list
        pre_idx = node_index[int(pre_id)]
        post_idx = node_index[int(post_id)]
        
        # get the actual neuron objects
        pre_neuron = neurons[pre_idx:pre_idx+1]
        post_neuron = neurons[post_idx:post_idx+1]
        
        # type aa
        weight_aa = aa.loc[pre_id, post_id]
        if weight_aa > 0:
            # connect with scaled weight and fixed delay
            nest.Connect(
                pre_neuron,
                post_neuron,
                syn_spec={"weight": 0.5 * weight_aa, "delay": 1.5}
            )
        
        # type ad
        weight_ad = ad.loc[pre_id, post_id]
        if weight_ad > 0:
            nest.Connect(
                pre_neuron,
                post_neuron,
                syn_spec={"weight": 0.5 * weight_ad, "delay": 1.5}
            )
        
        # type da
        weight_da = da.loc[pre_id, post_id]
        if weight_da > 0:
            nest.Connect(
                pre_neuron,
                post_neuron,
                syn_spec={"weight": 0.5 * weight_da, "delay": 1.5}
            )
        
        # type dd
        weight_dd = dd.loc[pre_id, post_id]
        if weight_dd > 0:
            nest.Connect(
                pre_neuron,
                post_neuron,
                syn_spec={"weight": 0.5 * weight_dd, "delay": 1.5}
            )

KeyboardInterrupt: 

In [20]:
# set up spike recording device and connect to all neurons
spike_recorder = nest.Create("spike_recorder")
nest.Connect(neurons, spike_recorder)

In [21]:
# select random neurons for stimulation
import random 

num_neurons_to_stimulate = 10
stimulated_indices = random.sample(range(N_neurons), num_neurons_to_stimulate)
stimulated_neurons = [neurons[i] for i in stimulated_indices]
stimulated_neurons

[NodeCollection(metadata=None, model=iaf_psc_alpha, size=1, first=6649),
 NodeCollection(metadata=None, model=iaf_psc_alpha, size=1, first=6460),
 NodeCollection(metadata=None, model=iaf_psc_alpha, size=1, first=7453),
 NodeCollection(metadata=None, model=iaf_psc_alpha, size=1, first=8603),
 NodeCollection(metadata=None, model=iaf_psc_alpha, size=1, first=6215),
 NodeCollection(metadata=None, model=iaf_psc_alpha, size=1, first=6095),
 NodeCollection(metadata=None, model=iaf_psc_alpha, size=1, first=7664),
 NodeCollection(metadata=None, model=iaf_psc_alpha, size=1, first=8517),
 NodeCollection(metadata=None, model=iaf_psc_alpha, size=1, first=7349),
 NodeCollection(metadata=None, model=iaf_psc_alpha, size=1, first=6648)]

In [27]:
#  set up a multimeter to record membrane potential from one neuron that were activated
multimeter = nest.Create("multimeter", params={"record_from": ["V_m"], "interval": 0.1})
nest.Connect(multimeter, stimulated_neurons[0][0:1])

In [28]:
# create a DC generator for external stimulation
dc = nest.Create("dc_generator", params={"amplitude": 400.0, "start": 50.0, "stop": 150.0})

In [37]:
# connect DC generator to each selected neuron
for i in range(len(stimulated_neurons)):
    nest.Connect(dc, stimulated_neurons[i][0:1])

In [38]:
# run the simulation for the specified time
sim_time = 200.0
nest.Simulate(sim_time)


Mar 16 22:08:01 NodeManager::prepare_nodes [Info]: 
    Preparing 8864 nodes for simulation.

Mar 16 22:08:01 SimulationManager::start_updating_ [Info]: 
    Number of local nodes: 8864
    Simulation time (ms): 200
    Number of OpenMP threads: 1
    Not using MPI

Mar 16 22:08:01 SimulationManager::run [Info]: 
    Simulation finished.


In [39]:
# print spike events recorded during simulation
spikes = nest.GetStatus(spike_recorder, "events")[0]
print(spikes)

{'senders': array([6460, 6649, 6095, 6215, 6648, 7349, 7453, 7664, 8517, 8603, 6460,
       6649, 6095, 6215, 6648, 6649, 7349, 7453, 7664, 8517, 8603, 6460,
       6649, 6095, 6215, 6460, 6648, 7349, 7453, 7664, 8517, 8603, 6649,
       6095, 6215, 6460, 6648, 7349, 7453, 7664, 8517, 8603, 6649, 6460,
       6095, 6215, 6648, 6649, 7349, 7453, 7664, 8517, 8603, 6460, 6649,
       6095, 6215, 6648, 7349, 7453, 7664, 8517, 8603, 6460, 6649, 6095,
       6215, 6648, 6649, 7349, 7453, 7664, 8517, 8603, 6460, 6460, 6649,
       6095, 6215, 6648, 7349, 7453, 7664, 8517, 8603, 6460, 6649, 6095,
       6215, 6648, 6649, 7349, 7453, 7664, 8517, 8603, 6460, 6095, 6215,
       6648, 6649, 7349, 7453, 7664, 8517, 8603, 6460, 6649, 6095, 6215,
       6460, 6648, 7349, 7453, 7664, 8517, 8603, 6649, 6460, 6095, 6215,
       6648, 6649, 7349, 7453, 7664, 8517, 8603, 6460, 6649, 6095, 6215,
       6648, 7349, 7453, 7664, 8517, 8603, 6460, 6649, 6095, 6215, 6648,
       6649, 7349, 7453, 7664, 8517, 86

In [43]:
print(nest.Models())

('ac_generator', 'aeif_cond_alpha', 'aeif_cond_alpha_astro', 'aeif_cond_alpha_multisynapse', 'aeif_cond_beta_multisynapse', 'aeif_cond_exp', 'aeif_psc_alpha', 'aeif_psc_delta', 'aeif_psc_delta_clopath', 'aeif_psc_exp', 'amat2_psc_exp', 'astrocyte_lr_1994', 'bernoulli_synapse', 'bernoulli_synapse_hpc', 'bernoulli_synapse_lbl', 'clopath_synapse', 'clopath_synapse_hpc', 'clopath_synapse_lbl', 'cm_default', 'cont_delay_synapse', 'cont_delay_synapse_hpc', 'cont_delay_synapse_lbl', 'correlation_detector', 'correlomatrix_detector', 'correlospinmatrix_detector', 'dc_generator', 'diffusion_connection', 'eprop_iaf_adapt_bsshslm_2020', 'eprop_iaf_bsshslm_2020', 'eprop_learning_signal_connection_bsshslm_2020', 'eprop_readout_bsshslm_2020', 'eprop_synapse_bsshslm_2020', 'eprop_synapse_bsshslm_2020_hpc', 'erfc_neuron', 'gamma_sup_generator', 'gap_junction', 'gauss_rate_ipn', 'gif_cond_exp', 'gif_cond_exp_multisynapse', 'gif_pop_psc_exp', 'gif_psc_exp', 'gif_psc_exp_multisynapse', 'ginzburg_neuron', 

/Users/adema/miniconda3/envs/cde_py39/lib/python3.9/site-packages/nest/lib/hl_api_helper.py:114: UserWarning:
Models is deprecated and will be removed in a future version of NEST.
Please use nest.node_models or nest.synapse_models instead!


In [40]:
# print voltage recordings from the multimeter
dmm = nest.GetStatus(multimeter)[0]
Vms = dmm["events"]["V_m"]
for vm in Vms:
    print(vm)

-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.0
-70.